In [1]:
from diffusers import StableDiffusionPipeline
import os
import os.path as path

home = os.getenv('HOME')
p = StableDiffusionPipeline.from_single_file(path.join(home, 'civitai', 'animatediff', 'majicmixRealistic_v7.safetensors'))

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["bos_token_id"]` will be overriden.
`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["eos_token_id"]` will be overriden.
/home/kiddos/.pyenv/versions/3.11.4/lib/python3.11/site-packages/transformers/models/clip/feature_extraction_clip.py:28: FutureWarning: The class CLIPFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use CLIPImageProcessor instead.
  warnings.warn(


In [2]:
import torch
from diffusers import MotionAdapter, AnimateDiffPipeline
from diffusers.schedulers import DPMSolverSinglestepScheduler
from diffusers.utils import export_to_gif
import os
import os.path as path

adapter = MotionAdapter.from_pretrained("guoyww/animatediff-motion-adapter-v1-5-2")
model_id = "runwayml/stable-diffusion-v1-5"
pipe = AnimateDiffPipeline.from_pretrained(
  model_id,
  vae=p.vae.to(torch.float16),
  unet=p.unet.to(torch.float16),
  tokenizer=p.tokenizer,
  motion_adapter=adapter,
  torch_dtype=torch.float16,
  use_safetensors=True,
)
scheduler = DPMSolverSinglestepScheduler.from_config(
  pipe.scheduler.config
)
pipe.scheduler = scheduler

# pipe.enable_sequential_cpu_offload()
pipe.enable_vae_slicing()
pipe.enable_model_cpu_offload()

The config attributes {'motion_activation_fn': 'geglu', 'motion_attention_bias': False, 'motion_cross_attention_dim': None} were passed to MotionAdapter, but are not expected and will be ignored. Please verify your config.json configuration file.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

The config attributes {'transformer_layers_per_block': 1, 'attention_head_dim': 8, 'class_embed_type': None, 'addition_embed_type': None, 'addition_time_embed_dim': None, 'upcast_attention': None, 'projection_class_embeddings_input_dim': None} were passed to UNetMotionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


In [3]:
prompt = '1girl, solo, cowboy shot, latex bodysuit, huge breasts, sagging breasts, pink nipples, 8K, masterpiece, best quality, ultra high res, highres, extremely detailed, extremely delicate and beautiful, cinematic lighting, studio lighting, mottled light and shadow, hourglass figure, pronounced feminine feature, busty, narrow waist, elegant glamor pose, award winning glamour photograph, modelshoot style, facing viewer, from font, looking at viewer,, , studio lighting,'
negative_prompt = 'makeup, lipstick,  loli, worst quality, lowres, watermark, deformed, distorted, disfigured, poorly drawn, bad anatomy, wrong anatomy, extra limb, missing limb, floating limbs, (mutated hands and fingers:1.4), disconnected limbs, mutation, mutated, ugly, disgusting, amputation'

generator = torch.Generator("cpu").manual_seed(410293809141)
output = pipe(
  prompt=prompt.strip(),
  negative_prompt=negative_prompt.strip(),
  num_frames=16,
  guidance_scale=7.0,
  num_inference_steps=30,
  generator=generator,
  clip_skip=2,
)
frames = output.frames[0]
export_to_gif(frames, "/tmp/animatediff.gif")

Token indices sequence length is longer than the specified maximum sequence length for this model (101 > 77). Running this sequence through the model will result in indexing errors
The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: [', award winning glamour photograph, modelshoot style, facing viewer, from font, looking at viewer,,, studio lighting,']


  0%|          | 0/30 [00:00<?, ?it/s]

'/tmp/animatediff.gif'

In [4]:
prompt = """
1girl,leogirl,tsurime,solo,(smiling,happy),(bright red lips,blush:0.8),huge breasts,(sagging breasts),braids,walking,pose,
(dreamflower,multicolored_background,A melancholic autumn scene in a vast flower field,a gentle breeze rustling through the dry grass,fallen leaves scattered among the flowers,a bittersweet atmosphere,a moment of quiet contemplation,Soft and warm color palette,delicate brushwork,evocative use of light and shadow,subtle details in the wilting flowers,high contrast,color contrast),
masterpiece,official art,best quality,high quality,highres,natural light,ray tracing,volumetric light,realistic,photorealistic,ultra highres,vivid,nostalgia,bokeh,
"""
negative_prompt = """
BeyondV3-neg,BadNegAnatomyV1-neg,NegfeetV2,ng_deepnegative_v1_75t,EasyNegative,badhandv4,rev2-badprompt,verybadimagenegative_v1.3,bhands-neg,bad anatomy,bad hands,watermark,text,(worst quality,low quality,normal quality:1.5),lowres,((monochrome)),((grayscale)),username,(2 girls,multiple girls),negative_hand-neg, mutated arm, mutated hand, incorrect anatomy
"""

output = pipe(
  prompt=prompt.strip(),
  negative_prompt=negative_prompt.strip(),
  num_frames=16,
  guidance_scale=7.0,
  num_inference_steps=30,
  generator=generator,
  clip_skip=2,
)
frames = output.frames[0]
export_to_gif(frames, "/tmp/animatediff2.gif")

The following part of your input was truncated because CLIP can only handle sequences up to 77 tokens: ['leaves scattered among the flowers, a bittersweet atmosphere, a moment of quiet contemplation, soft and warm color palette, delicate brushwork, evocative use of light and shadow, subtle details in the wilting flowers, high contrast, color contrast ), masterpiece, official art, best quality, high quality, highres, natural light, ray tracing, volumetric light, realistic, photorealistic, ultra highres, vivid, nostalgia, bokeh,']


  0%|          | 0/30 [00:00<?, ?it/s]

'/tmp/animatediff2.gif'